# Silver Layer - Limpeza e Normalização# Objetivo: Limpar e normalizar os dados da camada Bronze, criando tabelas dimensionais e fatos

In [1]:
import sysimport ossys.path.append(os.path.abspath('../src'))import pandas as pdimport numpy as npfrom pyspark.sql import SparkSessionfrom pyspark.sql import functions as Ffrom pyspark.sql.window import Windowfrom sqlalchemy import create_engine, textimport warningswarnings.filterwarnings('ignore')print(" Bibliotecas importadas")

Bibliotecas importadas

In [2]:
# Sparkspark = SparkSession.builder \.appName("Silver_Layer_Censo_2022") \.config("spark.driver.memory", "4g") \.getOrCreate()print(f" Spark {spark.version}")

/usr/local/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not foundSetting default log level to "WARN".To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).25/10/16 20:21:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable

Spark 3.5.1

# PostgreSQL

In [3]:
POSTGRES_HOST = os.getenv('POSTGRES_HOST', 'localhost')POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')POSTGRES_DB = os.getenv('POSTGRES_DB', 'etl_censo')POSTGRES_USER = os.getenv('POSTGRES_USER', 'etl_user')POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'etl_password')postgres_url = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"engine = create_engine(postgres_url, pool_pre_ping=True)jdbc_url = f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"connection_properties = {"user": POSTGRES_USER,"password": POSTGRES_PASSWORD,"driver": "org.postgresql.Driver"}print(" PostgreSQL configurado")

PostgreSQL configurado

In [4]:
# 1. Criar Dimensões

In [5]:
# Limpar dependências antes de recriar tabelasprint("🧹 Limpando dependências...")with engine.connect() as conn:# Desabilitar verificações de chave estrangeira temporariamenteconn.execute(text("SET session_replication_role = replica;"))# Dropar tabelas com dependênciastry:conn.execute(text("DROP TABLE IF EXISTS silver.fato_trabalho CASCADE;"))print(" silver.fato_trabalho removida")except:passtry:conn.execute(text("DROP TABLE IF EXISTS silver.dim_uf CASCADE;"))print(" silver.dim_uf removida")except:passtry:conn.execute(text("DROP TABLE IF EXISTS silver.dim_sexo CASCADE;"))print(" silver.dim_sexo removida")except:passtry:conn.execute(text("DROP TABLE IF EXISTS silver.dim_cor_raca CASCADE;"))print(" silver.dim_cor_raca removida")except:passtry:conn.execute(text("DROP TABLE IF EXISTS silver.dim_nivel_instrucao CASCADE;"))print(" silver.dim_nivel_instrucao removida")except:pass# Reabilitar verificações de chave estrangeiraconn.execute(text("SET session_replication_role = DEFAULT;"))conn.commit()print(" Dependências limpas")

🧹 Limpando dependências...silver.fato_trabalho removidasilver.dim_uf removidasilver.dim_sexo removidasilver.dim_cor_raca removidasilver.dim_nivel_instrucao removidaDependências limpas

In [6]:
# 1.1 Dimensão UF/Região

In [7]:
dim_uf_data = [('AC','N','Acre'),('AM','N','Amazonas'),('AP','N','Amapá'),('PA','N','Pará'),('RO','N','Rondônia'),('RR','N','Roraima'),('TO','N','Tocantins'),('AL','NE','Alagoas'),('BA','NE','Bahia'),('CE','NE','Ceará'),('MA','NE','Maranhão'),('PB','NE','Paraíba'),('PE','NE','Pernambuco'),('PI','NE','Piauí'),('RN','NE','Rio Grande do Norte'),('SE','NE','Sergipe'),('ES','SE','Espírito Santo'),('MG','SE','Minas Gerais'),('RJ','SE','Rio de Janeiro'),('SP','SE','São Paulo'),('PR','S','Paraná'),('RS','S','Rio Grande do Sul'),('SC','S','Santa Catarina'),('DF','CO','Distrito Federal'),('GO','CO','Goiás'),('MS','CO','Mato Grosso do Sul'),('MT','CO','Mato Grosso')]dim_uf = pd.DataFrame(dim_uf_data, columns=['uf', 'regiao', 'nome_uf'])dim_uf.to_sql('dim_uf', engine, schema='silver', if_exists='replace', index=False)print(f" silver.dim_uf: {len(dim_uf)} UFs")

silver.dim_uf: 27 UFs

# 1.2 Dimensão Sexo

In [8]:
dim_sexo_data = [('M', 'Masculino'),('F', 'Feminino')]dim_sexo = pd.DataFrame(dim_sexo_data, columns=['sexo_codigo', 'sexo_descricao'])dim_sexo.to_sql('dim_sexo', engine, schema='silver', if_exists='replace', index=False)print(f" silver.dim_sexo: {len(dim_sexo)} categorias")

silver.dim_sexo: 2 categorias

In [9]:
# 1.3 Dimensão Cor/Raça

In [10]:
dim_cor_raca_data = [('B', 'Branca'),('P', 'Parda'),('N', 'Preta'),('A', 'Amarela'),('I', 'Indígena')]dim_cor_raca = pd.DataFrame(dim_cor_raca_data, columns=['cor_raca_codigo', 'cor_raca_descricao'])dim_cor_raca.to_sql('dim_cor_raca', engine, schema='silver', if_exists='replace', index=False)print(f" silver.dim_cor_raca: {len(dim_cor_raca)} categorias")

silver.dim_cor_raca: 5 categorias

In [11]:
# 1.4 Dimensão Nível de Instrução

In [12]:
dim_nivel_data = [('0', 'Sem instrução'),('1', 'Fundamental incompleto'),('2', 'Fundamental completo'),('3', 'Médio incompleto'),('4', 'Médio completo'),('5', 'Superior incompleto'),('6', 'Superior completo'),('7', 'Pós-graduação')]dim_nivel = pd.DataFrame(dim_nivel_data, columns=['nivel_codigo', 'nivel_descricao'])dim_nivel.to_sql('dim_nivel_instrucao', engine, schema='silver', if_exists='replace', index=False)print(f" silver.dim_nivel_instrucao: {len(dim_nivel)} níveis")

silver.dim_nivel_instrucao: 8 níveis

In [13]:
# 2. Criar Fatos

In [14]:
# 2.1 Fato Trabalho/Rendimento

In [15]:
# Ler dados de rendimento do Bronzedf_trab = pd.read_sql("SELECT * FROM bronze.censo_trabalhadores", engine)# Validar e limpardf_trab = df_trab.dropna(subset=['uf', 'rendimento_mensal_nominal', 'populacao_ocupada'])df_trab = df_trab[df_trab['populacao_ocupada'] > 0]df_trab = df_trab[df_trab['rendimento_mensal_nominal'] >= 0]# Salvar na camada Silverdf_trab.to_sql('fato_trabalho', engine, schema='silver', if_exists='replace', index=False)print(f" silver.fato_trabalho: {len(df_trab):,} registros")

silver.fato_trabalho: 1,350 registros

In [16]:
# 2.2 Fato Crescimento Populacional

In [17]:
# Ler crescimento populacional do Bronzedf_cresc = pd.read_sql("SELECT * FROM bronze.crescimento_populacional", engine)# Calcular crescimento anual e taxadf_cresc = df_cresc.sort_values('ano_pesquisa')df_cresc['crescimento_anual'] = df_cresc['populacao_pessoas'].diff()df_cresc['taxa_crescimento'] = (df_cresc['crescimento_anual'] / df_cresc['populacao_pessoas'].shift(1) * 100).round(2)# Salvar na camada Silverdf_cresc.to_sql('fato_crescimento_populacional', engine, schema='silver', if_exists='replace', index=False)print(f" silver.fato_crescimento_populacional: {len(df_cresc)} registros")

silver.fato_crescimento_populacional: 13 registros

In [18]:
# 2.3 Fato Território

In [19]:
# Ler território do Bronzedf_terr = pd.read_sql("SELECT * FROM bronze.territorio", engine)# Salvar na camada Silverdf_terr.to_sql('fato_territorio', engine, schema='silver', if_exists='replace', index=False)print(f" silver.fato_territorio: {len(df_terr)} registros")

silver.fato_territorio: 1 registros

# 2.4 Fato Características do Domicílio

In [20]:
# Ler características dos domicílios do Bronzedf_carac = pd.read_sql("SELECT * FROM bronze.caracteristicas_domicilios", engine)# Salvar na camada Silverdf_carac.to_sql('fato_caracteristicas_domicilio', engine, schema='silver', if_exists='replace', index=False)print(f" silver.fato_caracteristicas_domicilio: {len(df_carac)} registros")

silver.fato_caracteristicas_domicilio: 4 registros

# Resumo Silver Layer

In [21]:
with engine.connect() as conn:result = conn.execute(text("""SELECT table_name FROM information_schema.tablesWHERE table_schema = 'silver' ORDER BY table_name"""))print("\n" + "="*70)print(" Resumo da camada SILVER")print("="*70)print(f"{'Tabela':<40} {'Registros':>15}")print("-"*70)total_reg = 0for row in result:count_result = conn.execute(text(f"SELECT COUNT(*) FROM silver.{row[0]}"))count = count_result.fetchone()[0]tipo = "DIM" if row[0].startswith('dim_') else "FATO"print(f"{row[0]:<40} {count:>15,} ({tipo})")total_reg += countprint("-"*70)print(f"{'TOTAL':<40} {total_reg:>15,}")print("="*70)

======================================================================Resumo da camada SILVER======================================================================Tabela Registros----------------------------------------------------------------------dim_cor_raca 5 (DIM)dim_nivel_instrucao 8 (DIM)dim_sexo 2 (DIM)dim_uf 27 (DIM)fato_caracteristicas_domicilio 4 (FATO)fato_crescimento_populacional 13 (FATO)fato_territorio 1 (FATO)fato_trabalho 1,350 (FATO)----------------------------------------------------------------------TOTAL 1,410======================================================================

In [22]:
print("\n" + "="*70)print(" SILVER LAYER CONCLUÍDA concluída")print("="*70)print("\n Estrutura criada:")print("\n DIMENSÕES:")print(" dim_uf (27 UFs + regiões)")print(" dim_sexo (2 categorias)")print(" dim_cor_raca (5 categorias)")print(" dim_nivel_instrucao (8 níveis)")print("\n FATOS:")print(" fato_trabalho (rendimento por UF/sexo/raça)")print(" fato_crescimento_populacional (histórico 1872-2022)")print(" fato_territorio (área e densidade)")print(" fato_caracteristicas_domicilio (saneamento, infraestrutura)")print("\n Próximo passo: Executar gold.ipynb para calcular indicadores")

======================================================================SILVER LAYER CONCLUÍDA concluída======================================================================Estrutura criada:DIMENSÕES:dim_uf (27 UFs + regiões)dim_sexo (2 categorias)dim_cor_raca (5 categorias)dim_nivel_instrucao (8 níveis)FATOS:fato_trabalho (rendimento por UF/sexo/raça)fato_crescimento_populacional (histórico 1872-2022)fato_territorio (área e densidade)fato_caracteristicas_domicilio (saneamento, infraestrutura)Próximo passo: Executar gold.ipynb para calcular indicadores

In [23]:
spark.stop()print(" Spark Session encerrada")

Spark Session encerrada